In [1]:
import pandas as pd
import numpy as np
import Levenshtein
import re
import unidecode


data = pd.read_csv('Banco de Súmulas - Sumulas.csv')

C:\Users\Carol Erthal\AppData\Local\Temp\ipykernel_21148\2494190213.py:8: DtypeWarning: Columns (8,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('Banco de Súmulas - Sumulas.csv')


In [2]:
# keep only single classes
classes = ['Formula Kite M', 'Formula Kite F', 'ILCA 7', 'IQFOIL Fem.', 'Formula Kite Fem.', 'IQFOIL Masc.','Formula Kite Masc.', 'ILCA 6', 'Ilca 7', 'Ilca 6']
data = data[data['Classe Vela'].isin(classes)]

# upper case all "Nome Competidor" 
data['Nome Competidor'] = data['Nome Competidor'].str.upper()

# unidecode all "Nome Competidor"
data['Nome Competidor'] = data['Nome Competidor'].apply(unidecode.unidecode)

# replace double spaces with single space
data['Nome Competidor'] = data['Nome Competidor'].str.replace('  ', ' ')

# get unique names
names = data['Nome Competidor'].unique()

In [3]:
# fix mixed up names
def are_names_same(str1, str2):
    # Split the strings into words, remove spaces, and sort the lists of words
    name1 = sorted(str1.split())
    name2 = sorted(str2.split())
    
    # Check if the sorted lists of words are equal
    return name1 == name2

In [4]:
# levenshtein distance

# create df where rows and columns are the names and the values are the levenshtein distance
names_comparison = np.array(np.meshgrid(names, names)).T.reshape(-1,2)

# create a third column with the levenshtein distance
names_comparison = pd.DataFrame(names_comparison, columns=['name', 'other_names'])
names_comparison['levenshtein_distance'] = names_comparison.apply(lambda row: Levenshtein.distance(row['name'], row['other_names']), axis=1)

In [5]:
# set thresholds
ld1 = names_comparison[names_comparison['levenshtein_distance'] == 1]
# ld1.to_excel('levenshtein_distance_1.xlsx')
ld2 = names_comparison[names_comparison['levenshtein_distance'] == 2]
# ld2.to_excel('levenshtein_distance_2.xlsx')
ld3 = names_comparison[names_comparison['levenshtein_distance'] == 3]

In [6]:
# create dict with the names that are similar (dont insert same pair twice)
similar_names = {}

for ld in [ld1, ld2, ld3]:
    for _, row in ld.iterrows():
        # if the name is already in the dict
        if row['name'] in similar_names:
            # if the other name is not in the list, add it
            if row['other_names'] not in similar_names[row['name']]:
                similar_names[row['name']].append(row['other_names'])
            # if it is, do nothing
            else:
                pass

        # if the name is not in the dict and the other name is
        elif row['other_names'] in similar_names.values():
            # if the name isnt on other_name key, add it
            if row['name'] not in similar_names[row['other_names']]:
                similar_names[row['other_names']].append(row['name'])
            # if it is, do nothing
            else:
                pass
            
        # if none of the names are in the dict, add them as a new key
        else:
            # check which appears the most in the data and add that one as the key
            if data[data['Nome Competidor'] == row['name']].shape[0] > data[data['Nome Competidor'] == row['other_names']].shape[0]:
                similar_names[row['name']] = [row['other_names']]
            else:
                similar_names[row['other_names']] = [row['name']]

In [7]:
# if two keys receive True from the are_names_same function, 
# add the second key and value to the first key's value list and delete the second key

# create a list for the keys that will be deleted
keys_to_delete = []

for key1 in similar_names:
    for key2 in similar_names:
        # if the keys are the same, skip
        if key1 == key2:
            pass
        elif are_names_same(key1, key2) and (key2 not in keys_to_delete) and (key1 not in keys_to_delete):
            print(key1, key2)
            # add key2 value to key1 value
            similar_names[key1].extend(similar_names[key2])
            # add key 2 to key 1 value
            similar_names[key1].append(key2)
            # add key 2 to keys to delete
            keys_to_delete.append(key2)

print(keys_to_delete)

# delete the keys that are in the keys_to_delete list
for key in keys_to_delete:
    del similar_names[key]

SERGEY KOMISSAROV KOMISSAROV SERGEY
ILIJA MARKOVIC MARKOVIC ILIJA
CLEMENS KUEBBER KUEBBER CLEMENS
DIMITRIS PAPADIMITRIOU PAPADIMITRIOU DIMITRIS
JEAN BAPTISTE BERNAZ BERNAZ JEAN BAPTISTE
WAN LI LI WAN
SI WANG WANG SI
LIN PLETIKOS PLETIKOS LIN
KIM PLETIKOS PLETIKOS KIM
MAXIME JONKER JONKER MAXIME
EWAN MCMAHON MCMAHON EWAN
RYAN LO LO RYAN
LUDVIG LINDQVIST LINDQVIST LUDVIG
ADONIS BOUGIOURIS BOUGIOURIS ADONIS
MATT WEARN WEARN MATT
['KOMISSAROV SERGEY', 'MARKOVIC ILIJA', 'KUEBBER CLEMENS', 'PAPADIMITRIOU DIMITRIS', 'BERNAZ JEAN BAPTISTE', 'LI WAN', 'WANG SI', 'PLETIKOS LIN', 'PLETIKOS KIM', 'JONKER MAXIME', 'MCMAHON EWAN', 'LO RYAN', 'LINDQVIST LUDVIG', 'BOUGIOURIS ADONIS', 'WEARN MATT']


In [8]:
# for every name in names, if using are_names_same function with the name and any of the keys in similar_names returns True,
# add the name to the value of that key

for name in names:
    for key in similar_names:
        if are_names_same(name, key) and (name not in similar_names[key]) and (key != name):
            similar_names[key].append(name)
similar_names

{'LUKAS WALTON-KEIM': ['LUKAS WALTON- KEIM'],
 'LUCAS PES FONSECA': ['LUCAS PE FONSECA'],
 'SIRIWIT PRANGSRI': ['SIRAWIT PRANGSRI'],
 'GEORGE MANEV': ['GEORGI MANEV'],
 'MAYA ASHKENAZI': ['MAYA ASSHKENAZI', 'ASHKENAZI MAYA'],
 'MARIE-EVE MAYRAND': ['MARIE EVE MAYRAND'],
 'JAN VOSTER': ['JAN VOESTER', 'VOSTER JAN'],
 'SERGEY KOMISSAROV': ['SERGEI KOMISSAROV',
  'KOMISSAROV SERGEI',
  'KOMISSAROV SERGEY'],
 'MAOR BEN HAROSH': ['MAOR BEN HROSH', 'BEN HAROSH MAOR'],
 'CHUSITT PUNJAMALA': ['CHUSIT PUNJAMALA', 'PUNJAMALA CHUSITT'],
 'THOMAS VAN OFWEGEN': ['THOMAS VAN OFWEGWN', 'VAN OFWEGEN THOMAS'],
 'ILIJA MARKOVIC': ['ILLIJA MARKOVIC', 'MARKOVI? ILIJA', 'MARKOVIC ILIJA'],
 'CLEMENS KUEBBER': ['CLEMENS KUBBER', 'KUBBER CLEMENS', 'KUEBBER CLEMENS'],
 'SEBASTIAN KOERDEL': ['SEBASTIAN KORDEL', 'KOERDEL SEBASTIAN'],
 'MATTHEW BARTON': ['MATHEW BARTON'],
 'RADOSLAW FURMANSKI': ['RADOSLAW FUMANSKI', 'FURMANSKI RADOSLAW'],
 'BYRON KOKKALANIS': ['VYRON KOKKALANIS'],
 'ANTONINO CANGEMI': ['ANTONIO C